In [102]:
import pandas as pd
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

In [49]:
from utils import get_tickers_from_market_index

idiv = get_tickers_from_market_index("./assets/IDIV-composition.csv")

len(idiv)

In [50]:
tickers_code = [f"{ticker}.SA" for ticker in idiv]
tickers_code

## Obter os dados de dividendos

In [51]:
import yfinance as yf

tickers = yf.Tickers(tickers_code).tickers

In [52]:
all_dividends = pd.DataFrame()

for ticker, data in tickers.items():
    per_month_df = data.actions.groupby(pd.Grouper(freq="M")).sum()
    dividend_df = per_month_df.iloc[:, 0].to_frame()
    dividend_df.rename(columns={'Dividends': ticker}, inplace=True)

    if all_dividends.empty:
        all_dividends = dividend_df
        continue

    all_dividends = all_dividends.merge(dividend_df, left_index=True, right_index=True, how='outer')
    all_dividends.fillna(0, inplace=True)

all_dividends.reset_index(inplace=True)
all_dividends.set_index("Date", inplace=True)
all_dividends.index = pd.to_datetime(all_dividends.index)

all_dividends

In [53]:
all_dividends.to_csv("./assets/idiv_companies_dividends.csv")

## ## Agrupamento com base no dividendo

Classificações:
- Dividendo alto
- Dividendo moderado
- Dividendo baixo

In [228]:
all_dividends = pd.read_csv("./assets/idiv_companies_dividends.csv")
all_dividends.set_index("Date", inplace=True)

In [229]:
import plotly.express as px

fig = px.line(all_dividends, x=all_dividends.index, y=all_dividends.columns[:5])
fig.update_layout(title="Dividendo das empresas listados em bolsa",
                  xaxis_title="Data",
                  yaxis_title="Valor")
fig.show()

In [230]:
total_dividend = all_dividends.sum().to_frame("Dividend")
total_dividend

In [231]:
px.box(total_dividend)

In [232]:
import numpy as np

mean = np.mean(total_dividend.Dividend)
std_dev = np.std(total_dividend.Dividend)

lower_fence = mean - 1.5 * std_dev
upper_fence = mean + 1.5 * std_dev

outliers = total_dividend[(total_dividend.Dividend < lower_fence) | (total_dividend.Dividend > upper_fence)]

total_dividend.drop(outliers.index, inplace=True)
total_dividend.sort_values(by="Dividend", inplace=True)
total_dividend

In [233]:
total_dividend.describe()

In [234]:
from sklearn.preprocessing import StandardScaler

scaller = StandardScaler()
total_dividend_scaled = scaller.fit_transform(total_dividend.to_numpy())
total_dividend_scaled

In [235]:
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=3, random_state=0)
cluster_model.fit(total_dividend_scaled)

In [236]:
centers = cluster_model.cluster_centers_
centers = scaller.inverse_transform(centers).ravel()
centers = sorted(centers)
centers

In [237]:
import numpy as np

labels = cluster_model.labels_
np.unique(np.array(labels), return_counts=True)

In [238]:
import plotly.graph_objects as go

classification_legend = [
    "Dividendos Baixo",  # 0
    "Dividendos Moderado",  # 1
    "Dividendos Alto"  # 2
]

graph1 = px.scatter(total_dividend, x=total_dividend.index, y=total_dividend.columns, color=labels)
graph2 = px.scatter(x=classification_legend, y=centers, size=[12, 12, 12])
graph3 = go.Figure(data=graph1.data + graph2.data)
graph3.show()

In [239]:
classification = np.column_stack((total_dividend, labels))
classification_df = pd.DataFrame(classification, columns=["Dividend", "Classification"])

In [240]:
classified_all_dividends = pd.merge(total_dividend, classification_df, on="Dividend", how="left")
classified_all_dividends.index = total_dividend.index
classified_all_dividends.sort_values(by="Dividend", ascending=True)

In [241]:
possible_classifications = len(classification_legend)
classified_tickers = [[""]] * possible_classifications

for i in range(possible_classifications):
    filtered_df = classified_all_dividends[classified_all_dividends["Classification"] == i]
    classified_tickers[i] = filtered_df.index.to_list()

classified_tickers

## Definir um CAP (Carrying capacity) médio em cada categoria de empresa pagadora de dividendos

In [242]:
cap_per_cluster = {
    classification: 0.0
    for classification in range(possible_classifications)
}

all_dividends.index = pd.to_datetime(all_dividends.index, utc=True)
all_dividends_by_year = all_dividends.groupby(by=pd.Grouper(freq="Y")).sum()

for classification, cluster in enumerate(classified_tickers):
    cluster_mean_cap = []

    for ticker in cluster:
        ticker_dividend_history_df = all_dividends_by_year[ticker]

        ticker_dividend_history_df = ticker_dividend_history_df.pct_change()
        ticker_dividend_history_df = ticker_dividend_history_df.replace([np.inf, -np.inf], pd.NaT).dropna()
        median_dividend = ticker_dividend_history_df.median()
        cluster_mean_cap.append(median_dividend)

    cap_per_cluster[classification] = np.mean(cluster_mean_cap)

cap_per_cluster

## Treinamento do modelo para cada classificação de dividendos

In [243]:
from prophet import Prophet

ts_models = []

for i, _ in enumerate(classification_legend):
    df = all_dividends[classified_tickers[i]].mean(axis=1).to_frame()
    df.reset_index(inplace=True)
    df.rename(columns={"Date": "ds", 0: "y"}, inplace=True)
    df["ds"] = pd.to_datetime(df["ds"], utc=True).dt.tz_convert(None)
    df["cap"] = cap_per_cluster[i]

    model = Prophet()
    model.fit(df)
    ts_models.append(model)

## Previsão de dividendos

In [244]:
for model in ts_models:
    future = model.make_future_dataframe(periods=12 * 5, freq="MS")
    forecast = model.predict(future)
    model.plot(forecast)

### Previsão de dividendos para o próximo ano da EGIE3.SA

In [245]:
ticker = "VALE3.SA"

In [246]:
dividends_df = all_dividends[ticker].to_frame()
dividends_df

In [247]:
total_dividend = dividends_df[ticker].to_numpy().sum()

payer_classification = cluster_model.predict(scaller.transform(total_dividend.reshape(-1, 1)))[0]
payer_classification

In [248]:
dividends_df.reset_index(inplace=True)
dividends_df.rename(columns={"Date": "ds", "EGIE3.SA": "y"}, inplace=True)
dividends_df["ds"] = pd.to_datetime(dividends_df["ds"], utc=True).dt.tz_convert(None)
dividends_df

In [249]:
model = ts_models[payer_classification]

future = model.make_future_dataframe(periods=12, freq="M")
forecast = model.predict(future)

In [250]:
# Substituir os dividendos com valores negativos por 0
forecast.loc[forecast["yhat"] < 0, "yhat"] = 0
forecast = forecast[["ds", "yhat"]]
forecast

In [251]:
forecast.rename(columns={"ds": "Date", "yhat": ticker}, inplace=True)
forecast

# Cálculo do valor intrínseco

In [252]:
forecast_pcr_change = forecast.set_index("Date")

In [253]:
forecast_pcr_change = forecast_pcr_change.groupby(by=pd.Grouper(freq="Y")).sum()
forecast_pcr_change = forecast_pcr_change.pct_change()
forecast_pcr_change.dropna(inplace=True)
forecast_pcr_change

$$
P = \frac{D_1}{r - g}
$$

Onde,
- P = Valor intrínseco da empresa;
- D1 = Valor a ser pago em dividendos no próximo ano;
- r = Custo constante de capital próprio para a empresa (ou taxa de retorno). Nesse caso usaremos o [ROIC](https://statusinvest.com.br/termos/r/roic) como indicador, porém é possível utilizar outras métricas como por exemplo o WACC;
- g = Taxa constante de crescimento dos dividendos. Nesse modelo utilizamos a mediana do valor pago anualmente de dividendos até o ano atual e 1 ano no futuro (preditivo).
    
> Vale resaltar que as empresas classificados como "Dividendos Baixo" não são contempladas pelo modelo, pois podem ser empresas small caps ou apenas não pagam dividendos. Dessa forma, as empresas classificadas como "Dividendos Moderado" é utilizado o $r$ como o ROIC atual, já aqueles rotulados como "Dividendos Alto", o valor $r$ é o ROIC médio.

In [254]:
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import time

driver = webdriver.Chrome()
driver.get(f"https://statusinvest.com.br/acoes/{ticker.replace('.SA', '')}")

roic_button = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located(
        (By.XPATH, r'//*[@id="indicators-section"]/div[2]/div/div[4]/div/div[3]/div/div/div/button'))
)
time.sleep(2)
roic_button.click()

average_roic_element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, r'//*[@id="main-modal"]/div[2]/div[1]/div/div[1]/div/div[1]/strong'))
)
average_roic = average_roic_element.text

currant_roic_element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, r'//*[@id="main-modal"]/div[2]/div[1]/div/div[1]/div/div[2]/strong'))
)
currant_roic = currant_roic_element.text

exit_roic_menu_button = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, r'//*[@id="main-modal"]/div[1]/button'))
)
time.sleep(2)
exit_roic_menu_button.click()

ev_element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, r'/html/body/main/div[5]/div[1]/div/div[2]/div[8]/div/div/strong'))
)
ev = ev_element.text

number_of_shares_element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((By.XPATH, r'/html/body/main/div[5]/div[1]/div/div[2]/div[9]/div/div/strong'))
)
number_of_shares = number_of_shares_element.text

driver.quit()

In [255]:
def parse(raw: str) -> float:
    return float(raw.split()[0].replace(".", "").replace(",", ".").replace("%", ""))


def get_classification_legend(classified_all_dividends_: pd.DataFrame):
    labels = ["Baixo", "Moderado", "Alto"]

    uniques = classified_all_dividends_["Classification"].unique().tolist()
    dividends = {}

    for classification_unique in uniques:
        dividend = classified_all_dividends_[classified_all_dividends_["Classification"] == classification_unique].iloc[0, 0]
        dividends.update({classification_unique: dividend})
    dividends = {key: value for key, value in sorted(dividends.items(), key=lambda item: item[1])}

    return {classification: labels[i] for i, classification in enumerate(dividends.keys())}


def get_classification_label_for(ticker: str, classified_all_dividends_: pd.DataFrame):
    legend = get_classification_legend(classified_all_dividends_)
    classification = classified_all_dividends_[classified_all_dividends_.index == ticker].iloc[0, -1]
    
    return legend[classification]


In [256]:
roic = parse(currant_roic) if get_classification_label_for(ticker, classified_all_dividends) == "Moderado" else parse(average_roic)
roic

In [264]:
def calculate_intrinsic_value():
    vi_current_roic = forecast.iloc[-12:, 1].sum() / (parse(currant_roic) / 100 - np.median(forecast_pcr_change))
    vi_average_roic = forecast.iloc[-12:, 1].sum() / (parse(average_roic) / 100 - np.median(forecast_pcr_change))
    vi_reference = parse(ev) / parse(number_of_shares)
    
    if round(vi_current_roic, 0) == round(vi_average_roic, 0):
        if vi_reference * 0.8 <= vi_current_roic <= vi_reference * 1.2:
            return vi_current_roic
        else:
            return 0
    
    if abs(vi_reference - vi_current_roic) > abs(vi_reference - vi_average_roic) and vi_reference * 0.8 <= vi_average_roic <= vi_reference * 1.2:
        return vi_average_roic
    elif 0.8 <= vi_current_roic <= vi_reference * 1.2:
        return vi_current_roic
    else:
        return 0
        

In [265]:
calculate_intrinsic_value()